In [1]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import wordpunct_tokenize
# from nltk import word_tokenize
# from nltk.stem.porter import *
# from nltk.tokenize import RegexpTokenizer
import os,sys
# import re
import pickle
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
# import random
# import math
# from collections import Counter
# import functools
import heapq
# import time , heapq
# from itertools import * #Functions creating iterators for efficient looping

In [2]:
def cosine(DOCx, DOCy):
    '''
    input: doc1 path name(str) , doc2 path name(str)
    ouput: two doc's cosine similarity
    '''
    dfx = pd.read_csv(DOCx,sep=' ',names=['tindex','tfidf'],header=None)
    dfy = pd.read_csv(DOCy,sep=' ',names=['tindex','tfidf'],header=None)
    dfx = dfx.drop(0)
    dfy = dfy.drop(0)
    dfxy = pd.merge(dfx,dfy,on='tindex',how='outer')
    dfxy.fillna(0,inplace=True)
    up = sum(dfxy.tfidf_x * dfxy.tfidf_y)
    down = np.sqrt(sum(np.square(dfxy.tfidf_x)))* np.sqrt(sum(np.square(dfxy.tfidf_y)))
    result = up / down
    return result

similarity measure between clusters

In [3]:
def merge_sim(C, j, i, m):
    a = np.max([C[j][i], C[j][m]]) #single link
    b = np.min([C[j][i], C[j][m]]) # complete link
    return (a+b)/2 # max: single，min:complete

In [4]:
def heap_merge_sim(C, j, i, m):
    a = np.min([C[j][i][0], C[j][m][0]])
    b = np.max([C[j][i][0], C[j][m][0]])
    return (a+b)/2#max 取不相似的 #min:取相似的(single)

Cosine similarity for pair-wise document similarity

In [5]:
# N=1095
# C=np.zeros([N,N])
# I=np.ones((N,), dtype=int)
# eps=1e-10
# for i in tqdm(range(N)):
#     for j in range(N-i-1):
#         cos = cosine('data/tf-idf/'+str(i+1)+'.txt','data/tf-idf/'+str(j+i+2)+'.txt')
#         C[i][j+i+1]=C[j+i+1][i]=cos+eps
# print(C.shape)
# pickle.dump(obj=C,file=open('data/C.pkl','wb'))

In [6]:
N=1095
I=np.ones((N,), dtype=int)
eps=1e-10

cluster_representations = []
simple_cluster_results = []
C = pickle.load(open('data/C.pkl','rb'))
Ks = [8, 13, 20]
for n in range(N):
    cluster_representations.append([n])

original HAC

In [7]:
# simple_HAC_stime = time.time()
simple_A = []
for k in tqdm(range(N - 1)):

    max_sim = 0
    max_i = 0
    max_m = 0
    for i in range(N): #去找最大相似度的df<i,m><-argmax
        for m in range(i + 1):
            if i != m and I[i] == 1 and I[m] == 1 and C[i][m] >= max_sim:
                max_sim = C[i][m]
                max_i = i
                max_m = m
                
    simple_A.append((max_i, max_m))

    cluster_representations[max_i] += cluster_representations[max_m] #把m並進去i
    cluster_representations[max_m] = None
    
    for j in range(N): #更新全部文章對i的similarity
        the_sim = merge_sim(C, j, max_i, max_m)
        C[max_i][j] = the_sim
        C[j][max_i] = the_sim
        
    I[max_m] = 0 #m已經合併
    
    if np.sum(I) in Ks: #如果現在還活著的個數有再想要的分群當中

        the_cluster_result = sorted([sorted(cluster) for cluster in cluster_representations if cluster is not None])
        simple_cluster_results.append(the_cluster_result)


# print('Simple HAC Time Taken:', time.time() - simple_HAC_stime)
simple_cluster_results

100%|██████████| 1094/1094 [04:15<00:00,  4.28it/s]


[[[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   29,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   42,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   68,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   83,
   84,
   86,
   87,
   88,
   89,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   100,
   102,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   113,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   123,
   124,
   125,
   126,
   127,
   128,
   182,
   225,
   231,
   258,
   630,
   631,
   648,
   663,
   687,
   688,
   711,
   728],
  [10,
   18,
   28,
   112,
   114,
   168,
   261,
   277,
   295,
   300,
   303,
   307,
   315,
   316,
   

HEAP HAC
* 只需要simple的三分之一時間

In [8]:
heap_cluster_results = []
C_ = pickle.load(open('data/C.pkl','rb'))
C = []
P = []
I = np.zeros(N)
cluster_representations = []
N=1095
# heap_HAC_stime = time.time()
for n in range(N):
    c = []
    for i in range(N):
        the_sim = -C_[n][i]     
        c.append([the_sim, i])
    C.append(c)
    I[n] = 1
    C_list = sorted(C[n])
    C_list.remove(C[n][n])
    P.append(C_list)
    heapq.heapify(P[n])
    cluster_representations.append([n])
heap_A = [] # list of merges
for k in tqdm(range(N - 1)):
    min_neg_sim = 0
    min_i = 0
    min_m = 0
    for i in range(N):
        if I[i] == 1 and P[i][0][0] <= min_neg_sim: #similarity 最大的
            min_i = i
            min_neg_sim = P[i][0][0]
    min_m = P[min_i][0][1]
    k2, k1 = sorted([min_i, min_m]) 
    heap_A.append((k1, k2))
    I[k2] = 0 #k2併去k1
    P[k1] = []
    for i in range(N):
        if I[i] == 1 and i != k1: #for each i with 
            P[i].remove(C[i][k1])
            P[i].remove(C[i][k2])
            heapq.heapify(P[i])
            the_sim = heap_merge_sim(C, i, k1, k2) #sim
            C[i][k1][0] = the_sim
            heapq.heappush(P[i], C[i][k1])
            C[k1][i][0] = the_sim
            heapq.heappush(P[k1], C[k1][i])
    cluster_representations[k1] += cluster_representations[k2]
    cluster_representations[k2] = None
    if np.sum(I) in Ks:
        the_cluster_result = sorted([sorted(cluster) for cluster in cluster_representations if cluster is not None])
        heap_cluster_results.append(the_cluster_result)
# print('Heap HAC_ Time Taken', time.time() - heap_HAC_stime)
heap_cluster_results

100%|██████████| 1094/1094 [01:26<00:00, 12.70it/s]


[[[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   29,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   42,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   68,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   83,
   84,
   86,
   87,
   88,
   89,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   100,
   102,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   113,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   123,
   124,
   125,
   126,
   127,
   128,
   182,
   225,
   231,
   258,
   630,
   631,
   648,
   663,
   687,
   688,
   711,
   728],
  [10,
   18,
   28,
   112,
   114,
   168,
   261,
   277,
   295,
   300,
   303,
   307,
   315,
   316,
   

In [9]:
82.96472764015198 / 250.30226469039917

0.33145815817036933

各種不同similarity link算法

In [10]:
simple_single = simple_cluster_results
heap_single = heap_cluster_results
len(simple_single[2]) , len(heap_single[2])

(8, 8)

In [11]:
simple_avg1 = simple_cluster_results #幾何平均數
heap_avg1 = heap_cluster_results
len(simple_avg1[2]) , len(heap_avg1[2])

(8, 8)

In [12]:
simple_avg2 = simple_cluster_results #算術平均數
heap_avg2 = heap_cluster_results
len(simple_avg2[2]) , len(heap_avg2[2])

(8, 8)

In [13]:
simple_complete = simple_cluster_results
heap_complete = heap_cluster_results
len(simple_complete[2]) , len(heap_complete[2])

(8, 8)

In [14]:
with open('./data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
train_dict #class:doc_id
# train_dict = pickle.load(open('data/train_dict.pkl','rb'))

{'1': ['11',
  '19',
  '29',
  '113',
  '115',
  '169',
  '278',
  '301',
  '316',
  '317',
  '321',
  '324',
  '325',
  '338',
  '341'],
 '2': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '12',
  '13',
  '14',
  '15',
  '16'],
 '3': ['813',
  '817',
  '818',
  '819',
  '820',
  '821',
  '822',
  '824',
  '825',
  '826',
  '828',
  '829',
  '830',
  '832',
  '833'],
 '4': ['635',
  '680',
  '683',
  '702',
  '704',
  '705',
  '706',
  '708',
  '709',
  '719',
  '720',
  '722',
  '723',
  '724',
  '726'],
 '5': ['646',
  '751',
  '781',
  '794',
  '798',
  '799',
  '801',
  '812',
  '815',
  '823',
  '831',
  '839',
  '840',
  '841',
  '842'],
 '6': ['995',
  '998',
  '999',
  '1003',
  '1005',
  '1006',
  '1007',
  '1009',
  '1011',
  '1012',
  '1013',
  '1014',
  '1015',
  '1016',
  '1019'],
 '7': ['700',
  '730',
  '731',
  '732',
  '733',
  '735',
  '740',
  '744',
  '752',
  '754',
  '755',
  '756',
  '757',
  '759',
  '760'],
 '8': ['262',
  '296',
  '30

In [15]:
df1 = pd.DataFrame.from_dict(train_dict)
df1 = df1.astype('int')
df1['2'].tolist()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16]

In [16]:
df = pd.read_csv('data/voting_rev.csv')
clf13=[]
for i in range(13):
    
    temp = df[df.Value == (i+1)]['id'].tolist()
    temp.extend(df1[str(i+1)].tolist())
    temp = sorted(temp)
    clf13.append(temp)
len(clf13)

13

voting

In [17]:
# pickle.dump(obj=voting_lis , file=open('data/voting_lis.pkl','wb'))
# pickle.dump(obj=clf13 , file=open('data/clf13.pkl','wb'))
voting_lis = pickle.load(open('data/voting_lis.pkl','rb'))
clf13 = pickle.load(open('data/clf13.pkl','rb'))

In [18]:
# voting_li8 = [ans_all , ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,ans_all,
#              simple_complete,simple_complete,simple_complete,heap_complete,heap_complete,heap_complete,
#              simple_single ,simple_avg1,heap_avg1,simple_avg2 ,heap_avg2] #=>8
# voting_li20 = [ans_all , ans_all,simple_complete,simple_complete,simple_complete,simple_complete,
#              heap_complete,heap_complete,heap_complete,heap_complete,simple_single ,
#              simple_avg1   ,heap_avg1,simple_avg2,heap_avg2 ] #20
# voting_li13 = [ans_all ,simple_complete,heap_complete,simple_single ,
#              simple_avg1,heap_avg1,simple_avg2, heap_avg2] #13
# voting_lis = [voting_li20,voting_li13,voting_li8]

In [19]:
# voting_li = [ans_all , ans_all , simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2 ] #3154/1613/4332
# voting_li = [ans_all , ans_all , ans_all,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2 ] #2073/2073/3929
# voting_li = [ans_all  ,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2 ] #/2095/3871
# voting_li = [ans_all,ans_all  ,simple_complete , heap_complete , simple_single , heap_single,
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #/1777/3730
# voting_li = [ans_all,ans_all  ,simple_complete , heap_complete ,
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #/1359/4221
# voting_li = [ans_all , ans_all , ans_all,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #3215/1449/4243
# voting_li = [ans_all , ans_all ,simple_complete , heap_complete , simple_single , 
#              simple_avg1 ,heap_avg1, simple_avg2,heap_avg2 ] #1925/1925/4046
# voting_li = [ans_all , ans_all, ans_all, ans_all, ans_all, ans_all, ans_all, 
#              simple_complete ,simple_complete ,simple_complete ,simple_complete ,
#              heap_complete ,heap_complete ,heap_complete ,heap_complete ,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2 ] #2230/3515=>578/1175
# voting_li = [ans_all , ans_all, ans_all, ans_all, ans_all, ans_all, 
#              simple_complete ,simple_complete ,simple_complete ,simple_complete ,
#              heap_complete ,heap_complete ,heap_complete ,heap_complete ,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2 ] #2226/3417=>20:574/1105
# voting_li = [ans_all , ans_all,ans_all,ans_all,
#              simple_complete    ,simple_complete,simple_complete,simple_complete,
#              heap_complete    ,heap_complete,heap_complete,heap_complete,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2 ] #2454/3384 =>20:683/1173
# voting_li = [ans_all ,
#              simple_complete    ,
#              heap_complete    ,
#              simple_single ,
#              simple_avg1   ,heap_avg1,
#              simple_avg2, heap_avg2] #2454/3384 =>20:683/1173

                #4: #5:/2062/3467，6://3592，7:/2125/3286

for i,voting_li in enumerate(voting_lis):
    pd8 = pd.DataFrame(0,index=range(1095),columns=range(8))
    pd13 = pd.DataFrame(0,index=range(1095),columns=range(13))
    pd20 = pd.DataFrame(0,index=range(1095),columns=range(20))
    for method in voting_li : #20,13,8
        sort20 = sorted(method[0],key=len)
        sort13 = sorted(method[1],key=len)
        sort8 = sorted(method[2],key=len)
        for c,cluster in enumerate(sort8): #8群
            for doc in cluster: #各群裡面成員
                pd8.loc[doc,c] +=1
        for c,cluster in enumerate(sort13):
            for doc in cluster:
                pd13.loc[doc,c] +=1
        for c,cluster in enumerate(sort20): #20群
            for doc in cluster: #各群裡面成員
                pd20.loc[doc,c] +=1
    sort13 = sorted(clf13,key=len)
    for c,cluster in enumerate(sort13):
        for doc in cluster:
            pd13.loc[doc,c] += int(len(voting_li)/1.3)
    if i ==0:
        df20 = pd20
        print(len(voting_li))
    elif i ==1:
        df13 = pd13
        print(len(voting_li))
    elif i == 2:
        df8 = pd8
        print(len(voting_li))

15
8
21


In [20]:
for df in [df8,df13,df20]:
    df['vote'] = 0
    for i in range(len(df)):
        df.loc[i,'vote'] = df.loc[i].idxmax()

In [21]:
counts = 0
counts_minimize = 0
for pd_ in [df8,df13,df20]:
    cols = pd_.columns.tolist()[:-1]
    count = 0
    count_minimize = 0
    for col in cols:
        count += len(pd_[pd_[col]>int(len(voting_li)/2 )])
    pd_2 = pd_.drop(['vote'],axis=1)
    for i in range(len(pd_2)):
        count_minimize += pd_2.loc[i].isin([pd_2.loc[i].max()]).sum()
    
    print(count,count_minimize)
    counts+=count
    counts_minimize += count_minimize
print("ALL:",counts,'Minimize:',counts_minimize)

697 1096
269 1095
381 1095
ALL: 1347 Minimize: 3286


In [22]:
for i in range(13):
    print(df13[df13['vote']==i].index.tolist())
print('======================================================================================')
for i in range(20):
    print(df20[df20['vote']==i].index.tolist())
print('======================================================================================')
for i in range(8):
    print(df8[df8['vote']==i].index.tolist())

[812, 816, 817, 818, 819, 820, 821, 823, 824, 825, 827, 828, 829, 831, 832, 833, 834, 836, 837, 842, 843, 846, 849, 850, 856, 857, 858, 861, 863, 865, 866, 867, 869, 871, 872, 873, 875, 877, 878, 880, 881, 890, 892, 908, 942, 943, 987, 1061]
[484, 519, 522, 525, 526, 528, 529, 530, 531, 535, 536, 537, 538, 539, 540, 543, 544, 545, 546, 547, 552, 553, 554, 555, 556, 557, 558, 562, 563, 564, 565, 566, 567, 568, 569, 571, 576, 578, 579, 586, 589, 591, 592, 593, 594, 595, 626, 640, 717, 737]
[30, 43, 69, 82, 85, 91, 99, 101, 304, 308, 314, 319, 325, 326, 327, 330, 333, 339, 343, 344, 350, 354, 357, 359, 361, 364, 369, 370, 374, 375, 379, 381, 390, 391, 392, 393, 394, 397, 402, 403, 407, 420, 423, 432, 435, 438, 439, 440, 518, 527, 590]
[10, 18, 28, 112, 114, 168, 277, 300, 315, 316, 320, 323, 324, 337, 340, 356, 368, 371, 376, 380, 382, 383, 385, 387, 388, 395, 399, 401, 404, 418, 421, 422, 424, 425, 428, 430, 434, 443, 450, 459, 463, 466, 467, 475, 476, 478, 481, 488, 493, 523, 542, 549, 

In [25]:
for df in [df20,df13,df8]:
    clus = df.columns.tolist()[:-1]
    f = open('output/'+str(len(clus))+'.txt','w')
    for clu_num in clus:
        for doc_id in df[df['vote']==clu_num].index.tolist(): #群集從小的開始寫入，最後一個cluster個數最多
            f.write(str(doc_id+1)+'\n')
#             f.write('\n')
        f.write('\n')
    f.close()

In [24]:
df8

,0,1,2,3,4,5,6,7,vote
0,0,0,0,0,10,3,5,3,4
1,0,0,0,0,10,3,5,3,4
2,0,0,0,0,10,3,5,3,4
3,0,0,0,0,10,3,5,3,4
4,0,0,0,0,10,3,5,3,4
5,0,0,0,0,10,3,5,3,4
6,0,0,0,0,10,3,5,3,4
7,0,0,0,0,10,3,5,3,4
8,0,0,0,0,10,3,5,3,4
9,0,0,0,0,10,3,5,3,4
